In [27]:
import requests,shutil
import numpy as np
from PIL import Image
import matplotlib.image as mpimg
from pyproj import Proj, transform
from osgeo import gdal,osr
from shapely.geometry import shape

In [3]:
url = "http://gdis.mgb.gov.ph/arcgis/rest/services/Flood_Landslide_Susceptibility_10K_new_web/MapServer/export?dpi=96&transparent=true&format=png8&bbox=13768453.20379189%2C1138715.818918207%2C13779699.001578331%2C1147563.404942204&bboxSR=102100&imageSR=102100&size=1177%2C926&f=image"

In [4]:
url.split('bbox=')[1].split('&bboxSR')[0].split('%2C')

['13768453.20379189',
 '1138715.818918207',
 '13779699.001578331',
 '1147563.404942204']

In [5]:
# Download png
impath = "file.png"
r = requests.get(url,stream=True)
with open(impath,'wb') as im:
    r.raw.decode_content = True
    shutil.copyfileobj(r.raw, im)
#     for chunk in r.iter_content(1024):
#         im.write(chunk)

In [6]:
def reproj(x,y):
    inProj = Proj(init='epsg:3857')
    outProj = Proj(init='epsg:4326')
    x2,y2 = transform(inProj,outProj,x,y)
    return x2,y2

In [7]:
# Get bbounds
def getBounds(url):
    bbox = url.split('bbox=')[1].split('&bboxSR')[0].split('%2C')
    to_float = list(map(lambda x: float(x),bbox))
    bbox = []
    for i in [0,2]:
        x,y = reproj(to_float[i],to_float[i+1])
        bbox.extend([x,y])
    return bbox
bounds = getBounds(url)
print(bounds)

[123.68411951650647, 10.175345268610686, 123.78514223684324, 10.253564805628109]


In [33]:
def arrayToRaster(array,fileName,EPSGCode,xMin,xMax,yMin,yMax,numBands):
    xPixels = array.shape[1]  # number of pixels in x
    yPixels = array.shape[0]  # number of pixels in y
    pixelXSize =(xMax-xMin)/xPixels # size of the pixel in X direction     
    pixelYSize = -(yMax-yMin)/yPixels # size of the pixel in Y direction

    driver = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(fileName,xPixels,yPixels,numBands,gdal.GDT_Byte, options = [ 'PHOTOMETRIC=RGB' ])
    dataset.SetGeoTransform((xMin,pixelXSize,0,yMax,0,pixelYSize))  

    datasetSRS = osr.SpatialReference()
    datasetSRS.ImportFromEPSG(EPSGCode)
    dataset.SetProjection(datasetSRS.ExportToWkt())
    
    for i in range(0,numBands):
        if i == 0:
            dataset.GetRasterBand(1).WriteArray(array[:,:,i])
        else:
            dataset.GetRasterBand(i+1).WriteArray(array[:,:,i])

    dataset.FlushCache()  # Write to disk.

In [34]:
# Read array into function
# img = np.asarray(Image.open(impath))
img = mpimg.imread(impath)
fname = "OUTPUT.TIF"
epsg = 4326
xmin,ymin,xmax,ymax = bounds[0],bounds[2],bounds[1],bounds[3]
bands = 1
arrayToRaster(img,fname,epsg,xmin,xmax,ymin,ymax,bands)

None


In [35]:
# Vars
array = img
xMin,xMax,yMin,yMax = xmin,xmax,ymin,ymax
numBands = bands
EPSGCode = epsg
fileName = fname

xPixels = array.shape[1]  # number of pixels in x
yPixels = array.shape[0]  # number of pixels in y
pixelXSize =(xMax-xMin)/xPixels # size of the pixel in X direction     
pixelYSize = -(yMax-yMin)/yPixels # size of the pixel in Y direction

driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(fileName,xPixels,yPixels,numBands,gdal.GDT_Byte, options = [ 'PHOTOMETRIC=RGB' ])
dataset.SetGeoTransform((xMin,pixelXSize,0,yMax,0,pixelYSize))  

datasetSRS = osr.SpatialReference()
datasetSRS.ImportFromEPSG(EPSGCode)
dataset.SetProjection(datasetSRS.ExportToWkt())

dataset.GetRasterBand(1).WriteArray(array[:,:,0])
dataset.FlushCache()  # Write to disk.

In [29]:
img.shape

(926, 1177, 4)

In [102]:
# To shp
import rasterio
from rasterio.features import shapes
mask = None
# with rasterio.drivers():
with rasterio.open(fname) as src:
    image = src.read(1) # first band
    results = (
    {'properties': {'raster_val': v}, 'geometry': s}
    for i, (s, v) 
    in enumerate(
        shapes(image, mask=mask, transform=src.transform)))

In [103]:
geoms = list(results)

In [104]:
# Filter 0 values
final = []
for geo in geoms:
    rast_val = geo['properties']['raster_val']
    if rast_val != 0.0:
        final.append(geo)

In [105]:
import geopandas as gp

[{'properties': {'raster_val': 1.0},
  'geometry': {'type': 'Polygon',
   'coordinates': [[(27.341498043568578, 10.253564805628109),
     (27.341498043568578, 10.376169100910198),
     (27.245058983035108, 10.376169100910198),
     (27.245058983035108, 10.498773396192288),
     (27.148619922501638, 10.498773396192288),
     (27.148619922501638, 10.621377691474379),
     (26.955741801434698, 10.621377691474379),
     (26.955741801434698, 10.743981986756468),
     (26.762863680367758, 10.743981986756468),
     (26.762863680367758, 10.866586282038558),
     (26.569985559300818, 10.866586282038558),
     (26.569985559300818, 10.989190577320647),
     (26.377107438233864, 10.989190577320647),
     (26.377107438233864, 11.111794872602738),
     (26.087790256633454, 11.111794872602738),
     (26.087790256633454, 11.234399167884828),
     (25.702034014499574, 11.234399167884828),
     (25.702034014499574, 11.357003463166917),
     (25.41271683289915, 11.357003463166917),
     (25.4127168328991

In [106]:
poly  = gp.GeoDataFrame.from_features(final)
poly.crs = {'init' :'epsg:3857'}

In [110]:
# Reproject
poly = poly.to_crs({'init': 'epsg:4326'})
# Save
poly.to_file('output.shp')